In [ ]:

"I've been sleeping around"
"Bob is suspicious that Tim and Allen are going to tell his wife that he's been sleeping around"
"Did you know that Tom is sleeping with Jerry's wife?"
"I already told you, you're not getting that promotion!"
"Why don't you care about me?"
"The man who through the ball out the window was calling for you."


In [ ]:
# Reload the dependencies
%reload_ext autoreload
%autoreload 2

from dataclasses import dataclass, field
from datetime import datetime
from typing import List
import sys
sys.path.insert(0, "../..")

from src.core.Agent import Agent
from src.core.ChatMessage import ChatMessage
from src.core.Constants import Role, Llm

from src.utils import Logger
from src.utils.Logger import Level

Logger.set_level(Level.DEBUG)

@dataclass
class Entity:
    id: str
    blurb: str
    embedding: List[float]

@dataclass
class Claim:
    id: str
    subject: str
    object: str
    valid_start: datetime
    valid_end: datetime
    embedding: List[float]

@dataclass
class ClaimEdge:
    id: str
    from_claim_id: str
    to_claim_id: str
    rel_type: str
    valid_start: datetime
    valid_end: datetime

@dataclass
class Facet:
    blurb: str
    embedding: List[float]
    support_claim_ids: List[str]
    embedding: List[float]

@dataclass
class Profile:
    id: str
    blurb: str
    support_claim_ids: List[str]
    embedding: List[float]

Raw response from LLM: {
    "subject": "Bob",
    "object": "Tom",
    "predicate": "thinks Tom is a good person"
}
EntityIdentificationResponse(subject='Bob', object='Tom', predicate='thinks Tom is a good person')
Raw response from LLM: {
    "subject": "Tom",
    "object": "Bob",
    "predicate": "is punched by"
}
EntityIdentificationResponse(subject='Tom', object='Bob', predicate='is punched by')
Bob punched Tom.


In [26]:
model = Llm.gpt_4o_mini

@dataclass
class EntityIdentificationResponse:
    entities: str = field(metadata={"desc": "A list of entities in the text."})


parsing_agent = Agent(
    system_prompt="You are a text parser. You will be given a message and you will need to identify components of the text.",
    response_type=EntityIdentificationResponse,
    llm_model=model)


response = parsing_agent.chat_with_message("Bob thinks Tom is a good person.")
print(response)

response = parsing_agent.chat_with_message("Tom is punched by Bob.")
print(response)

response = parsing_agent.chat_with_message("Do you remember that guy who threw the ball out the window?")
print(response)



Raw response from LLM: {
    "entities": "Bob; Tom"
}
EntityIdentificationResponse(entities='Bob; Tom')
Raw response from LLM: {
    "entities": "Tom; Bob"
}
EntityIdentificationResponse(entities='Tom; Bob')
Raw response from LLM: {
    "entities": "guy; ball; window"
}
EntityIdentificationResponse(entities='guy; ball; window')


In [ ]:
reorder_agent = Agent(
    system_prompt="You are a sentence reorderer. You will be given a sentence and you will need to reorder it such that the subject is at the beginning of the sentence and the object is at the end of the sentence.",
    response_type=None,
    llm_model=model)

reordered = reorder_agent.chat_with_message("Tom is punched by Bob.")
print(reordered)

In [24]:
@dataclass
class ClaimExtractionResponse:
    claims: List[str] = field(metadata={"desc": "A list of truths that the text assumes or claims to be true."})

claim_extractor = Agent(
    system_prompt="You are a claim extractor. You will be given a text and you will need to extract the claims.",
    response_type=ClaimExtractionResponse,
    llm_model=Llm.gpt_4o_mini)

response = claim_extractor.chat_with_message("Bob is suspicious that Tim and Allen are going to tell his wife that he's been sleeping around")
print(response)


Raw response from LLM: {
    "claims": [
        "Bob is suspicious about Tim and Allen's intentions.",
        "Bob believes Tim and Allen might inform his wife about his infidelity."
    ]
}
ClaimExtractionResponse(claims=["Bob is suspicious about Tim and Allen's intentions.", 'Bob believes Tim and Allen might inform his wife about his infidelity.'])
